<a href="https://colab.research.google.com/github/SMahabeer/ICE-Task-2/blob/main/ICE_Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sachin Mahabeer (st10053116) PDAN8412 ICE Task 2

The code below was taken/adapted from multiple GeeksforGeeks sources found in the reference list and enhanced by Claude (2025)

In [6]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, count, desc, split, explode
import findspark
from pyspark.sql.functions import regexp_extract, trim
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql.types import *
import matplotlib.pyplot as plt
import tarfile
import os
import pickle
import numpy as np
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [5]:
findspark.init()
spark = SparkSession.builder.appName("CNN").getOrCreate()
spark

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

base_dir = '/content/'
train_dir = os.path.join(base_dir, 'seg_train/seg_train')
test_dir = os.path.join(base_dir, 'seg_test/seg_test')

IMG_HEIGHT = 150
IMG_WIDTH = 150
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

class_names = list(train_generator.class_indices.keys())
print("Class Names:", class_names)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
Class Names: ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [8]:
underfit_model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(6, activation='softmax')
])

underfit_model.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

underfit_model.summary()

history_underfit = underfit_model.fit(
    train_generator,
    epochs=3,
    validation_data=test_generator
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 87616)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6)              │       525,702 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 526,150 (2.01 MB)

 Trainable params: 526,150 (2.01 MB)

 Non-trainable params: 0 (0.00 B)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/3
439/439 ━━━━━━━━━━━━━━━━━━━━ 178s 402ms/step - accuracy: 0.5447 - loss: 2.0216 - val_accuracy: 0.6837 - val_loss: 0.8909
Epoch 2/3
439/439 ━━━━━━━━━━━━━━━━━━━━ 183s 361ms/step - accuracy: 0.7765 - loss: 0.6527 - val_accuracy: 0.6947 - val_loss: 0.8828
Epoch 3/3
439/439 ━━━━━━━━━━━━━━━━━━━━ 156s 355ms/step - accuracy: 0.8628 - loss: 0.4206 - val_accuracy: 0.7063 - val_loss: 0.8545


In [ ]:
overfit_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dense(6, activation='softmax')
])

overfit_model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

overfit_model.summary()

history_overfit = overfit_model.fit(
    train_generator,
    epochs=25,
    validation_data=test_generator
)

In [ ]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping

final_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')
])

final_model.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

final_model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history_final = final_model.fit(
    train_generator,
    epochs=15,
    validation_data=test_generator,
    callbacks=[early_stopping]
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

predictions = final_model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

true_classes = test_generator.classes

print("Classification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_names))

cm = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix for Final Model')
plt.show()

In [ ]:
def plot_history(histories, titles):
    plt.figure(figsize=(18, 5))

    for i, (history, title) in enumerate(zip(histories, titles)):
        plt.subplot(1, 3, i + 1)
        plt.plot(history.history['accuracy'], label='Train Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title(f'{title} - Accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(loc='lower right')

    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(18, 5))
    for i, (history, title) in enumerate(zip(histories, titles)):
        plt.subplot(1, 3, i + 1)
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title(f'{title} - Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(loc='upper right')

    plt.tight_layout()
    plt.show()

plot_history(
    [history_underfit, history_overfit, history_final],
    ['Underfitting Model', 'Overfitting Model', 'Final Model']
)

In [ ]:
misclassified_indices = np.where(predicted_classes != true_classes)[0]

test_filepaths = test_generator.filepaths

plt.figure(figsize=(15, 10))
for i, idx in enumerate(misclassified_indices[:5]):
    plt.subplot(2, 3, i + 1)
    img = tf.keras.preprocessing.image.load_img(test_filepaths[idx], target_size=(IMG_HEIGHT, IMG_WIDTH))
    plt.imshow(img)
    plt.title(f"Actual: {class_names[true_classes[idx]]}\nPredicted: {class_names[predicted_classes[idx]]}")
    plt.axis('off')
plt.tight_layout()
plt.show()

**According to GeeksforGeeks and enhanced by Gemini (2025)**:

Why did the simple model underfit the data?
The simple model underfit because it lacked the capacity to learn the complex patterns in the image data. With only one convolutional layer and a small number of filters (16), it couldn't extract enough meaningful features (like textures, shapes, and object parts) to distinguish between the six different classes. As a result, both its training and validation accuracy remained low, indicating it couldn't even perform well on the data it was trained on.

Why did the complex model overfit?
The complex model overfit because it had too much capacity and was trained for too long without any regularization. The deep architecture with many layers and neurons allowed it to essentially memorize the training data, including its noise and random fluctuations, rather than learning the general underlying patterns. This is visible in the training graphs where the training accuracy soared towards 100%, while the validation accuracy stagnated or even decreased, creating a large gap between the two curves.

What role does Dropout play in preventing overfitting?
Dropout is a regularization technique that helps prevent overfitting by randomly setting a fraction of neuron activations to zero during each training update. This has two key effects:

Reduces Co-adaptation: It prevents neurons from becoming overly reliant on each other. Since any neuron can be "dropped out," the network is forced to learn more robust and redundant features.

Ensemble Effect: It can be thought of as training a large number of different, smaller neural networks. The final predictions are an average from this "ensemble," which generally leads to better generalization.

How does the number of convolutional layers affect feature learning?
The number of convolutional layers determines the hierarchy and complexity of features the model can learn.

Shallow Networks (Few Layers): Learn simple, low-level features like edges, corners, and color gradients.

Deep Networks (Many Layers): Build upon the features from previous layers to learn more complex, high-level features. For example, a first layer might detect edges, a second might combine them to find textures or simple shapes (like an eye or a wheel), and a third might combine those to identify objects (like a face or a car). More layers allow the model to understand the image in a more abstract and semantically rich way.

How does the number of epochs impact model performance?
An epoch is one complete pass through the entire training dataset. The number of epochs is a critical hyperparameter:

Too Few Epochs: Leads to underfitting. The model doesn't get enough opportunities to learn from the data, and its performance will be poor on both training and test sets.

Too Many Epochs: Can lead to overfitting. After the model has learned the general patterns, it may start memorizing the noise in the training data, causing its performance on unseen data (validation set) to degrade. This is why techniques like EarlyStopping are so valuable—they stop the training process when performance on the validation set stops improving.

What are the benefits of a confusion matrix over just looking at accuracy?
While accuracy gives a single, overall score of performance, a confusion matrix provides a much more detailed and insightful view. Its benefits include:

Per-Class Performance: It shows how well the model is doing for each individual class, revealing if the model is biased or struggling with specific categories.

Error Analysis: It details the types of errors being made. For example, we can see if the model frequently mistakes 'sea' for 'glacier' or 'street' for 'buildings'. This information is invaluable for model improvement.

Identifying Class Imbalance Issues: If one class has many more samples than others, a high accuracy score can be misleading. A confusion matrix reveals if the model is simply predicting the majority class all the time.

Calculating Advanced Metrics: It's the basis for calculating other important metrics like Precision, Recall, and F1-score for each class, which provide a more nuanced understanding of performance than accuracy alone.

## Reference list:

GeeksforGeeks, 2023. Confusion matrix in machine learning. [online] Available at: https://www.geeksforgeeks.org/confusion-matrix-machine-learning/ [Accessed 29 August 2025].

GeeksforGeeks, 2024. Difference between batch, epoch, and iteration in neural network. [online] Available at: https://www.geeksforgeeks.org/difference-between-batch-epoch-and-iteration/ [Accessed 29 August 2025].

GeeksforGeeks, 2024. Dropout in neural networks. [online] Available at: https://www.geeksforgeeks.org/dropout-in-neural-networks/ [Accessed 29 August 2025].

GeeksforGeeks, 2024. Introduction to convolutional neural network. [online] Available at: https://www.geeksforgeeks.org/introduction-to-convolutional-neural-network/ [Accessed 29 August 2025].

GeeksforGeeks, 2023. Underfitting and overfitting in machine learning. [online] Available at: https://www.geeksforgeeks.org/underfitting-and-overfitting-in-machine-learning/ [Accessed 29 August 2025].